In [1]:
# Data load part

import pandas as pd
import glob
import json

import datetime

import pdb

def return_date_and_time(datetime_count):
    origin = datetime.datetime(1970, 1, 1)
    
    delta = datetime.timedelta(seconds=datetime_count)
    current = origin + delta

    return current
     
json_files = glob.glob("./dataset/car-location-logs/2020-09-*.json")

pos_mat = []
time_vec = []
for json_file in json_files:
    with open(json_file) as f:
        df = json.load(f)
        items = df['Items']
        for item in items:
            pos_mat.append([item['lat'], item['lng']])
            time_vec.append(item['timestamp'])
                        
order_df = pd.read_csv("./dataset/order_query_202009.csv")

In [15]:
tmp = return_date_and_time(time_vec[0])
print(tmp)

2020-09-29 10:18:23


In [41]:
print(order_df.iloc[0,:])

ID                                            1126
Aboard Ing Address                  沖縄市高原６丁目１１−２付近
Aboard Ing Lat                              26.319
Aboard Ing Lng                             127.827
Approaching At                                 NaN
Arriving At                                    NaN
Charge ID                                      NaN
Created At                    2020-09-01T11:07:54Z
Deleted At                    2020-09-01T11:08:00Z
Destination                               沖縄自動車道付近
Destination Lat                            26.3107
Destination Lng                            127.788
Done At                                        NaN
Driving At                                     NaN
Is Notified Before Pick Up                   False
Label                                         代行予約
Manager ID                                     NaN
Order Able ID                                  375
Order Able Type                App\Entities\Client
Pick Up At                     

In [2]:
def convert_date_to_timestamp(date_string):
    year_string = date_string.split("T")[0]
    time_string = date_string.split("T")[1]
    time_string = time_string.split("Z")[0]
    year, month, day = year_string.split("-")
    time, minute, second = time_string.split(":")
    
    origin = datetime.datetime(1970, 1, 1)
    delta = datetime.datetime(year=int(year), month=int(month), day=int(day), hour=int(time), minute=int(minute), second=int(second)) - origin
    
    return 24*3600*delta.days + delta.seconds

In [47]:
import numpy as np
import matplotlib.pyplot as plt

pos_mat = np.array(pos_mat)
time_vec = np.array(time_vec)
order_time_vec = np.array([convert_date_to_timestamp(order_df.iloc[i,7]) for i in range(len(order_df))])

dist_vec = []
for i in range(len(order_time_vec)):
    order_time = order_time_vec[i]
    delta_vec = time_vec - order_time
    date_idx = np.where((0 <= delta_vec) & (delta_vec <= 5))
    if len(date_idx[0]) != 0:
        car_pos_mat = pos_mat[date_idx]
        order_pos = np.array([order_df.iloc[i,2], order_df.iloc[i,3]])
        delta_pos = np.array([x - order_pos for x in car_pos_mat])
        tmp_pos = delta_pos*100
    
        tmp_dist_vec = np.array([np.sqrt(x[0]**2 + x[1]**2) for x in tmp_pos])
        dist_vec.append(tmp_dist_vec)
    

In [65]:
d_count = np.zeros(10)

for d in dist_vec:
    for i in range(10):
        if 0 < len(np.where(d < i+1)[0]):
            d_count[i] += 1
ratio = np.array(d_count)/len(dist_vec)
print(np.diff(ratio))

[0.23899371 0.14465409 0.08490566 0.02515723 0.01257862 0.00628931
 0.00314465 0.         0.00628931]
